
# Financial Instruments Comprehensive Solutions

This notebook walks through the homework tasks sequentially with data ingestion, calculations, and Plotly visualizations.


In [1]:

import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

pd.set_option('display.max_columns', None)

def libor_simple_to_continuous(rate_percent, tenor_years):
    rate = rate_percent / 100.0
    return math.log(1 + rate * tenor_years) / tenor_years

def forward_fx(spot, r_dom_cc, r_for_cc, tenor_years):
    return spot * math.exp((r_dom_cc - r_for_cc) * tenor_years)


## Homework 1 – Forward Rates and Covered Parity

In [2]:

# Q1: One-year forward USD/EUR
spot = 1.20
r_us_cc = 0.05
r_eu_cc = 0.045
f_theoretical = forward_fx(spot, r_us_cc, r_eu_cc, 1)
print(f"No-arbitrage 1Y forward: {f_theoretical:.5f} USD/EUR")

k_market = 1.15
value_diff = f_theoretical - k_market
print(f"Market forward discount vs fair: {value_diff:.5f}")

if k_market < f_theoretical:
    arbitrage = "Forward underpriced: borrow USD, convert to EUR, invest EUR, and go long forward to lock cheap dollars later."
else:
    arbitrage = "Forward overpriced: borrow EUR, convert to USD, invest USD, and short the forward."
print(arbitrage)


No-arbitrage 1Y forward: 1.20602 USD/EUR
Market forward discount vs fair: 0.05602
Forward underpriced: borrow USD, convert to EUR, invest EUR, and go long forward to lock cheap dollars later.


In [3]:

# Q2: Covered interest parity using DataHW1
hw1_path = Path('Assignments/Assignment 1/DataHW1.xls')
raw = pd.read_excel(hw1_path, sheet_name='FX_Forwards_and_Rates', header=0)
clean = raw.iloc[1:].copy()
clean.columns = ['Date','Spot','Fwd1M','Fwd3M','Fwd6M','Fwd1Y','US1M','US3M','US6M','US1Y','EU1M','EU3M','EU6M','EU1Y']
clean['Date'] = pd.to_datetime(clean['Date'], errors='coerce')
clean = clean.dropna(subset=['Date'])

maturities = {'Fwd1M':1/12,'Fwd3M':3/12,'Fwd6M':6/12,'Fwd1Y':1}
records = []
for _, row in clean.iterrows():
    spot = row['Spot']
    for col, tenor in maturities.items():
        r_us = libor_simple_to_continuous(row[f'US{int(tenor*12)}M' if tenor<1 else 'US1Y'], tenor)
        r_eu = libor_simple_to_continuous(row[f'EU{int(tenor*12)}M' if tenor<1 else 'EU1Y'], tenor)
        theo = forward_fx(spot, r_us, r_eu, tenor)
        quoted = row[col]
        records.append({
            'Date': row['Date'],
            'Tenor': col.replace('Fwd',''),
            'Theoretical': theo,
            'Quoted': quoted,
            'Deviation': quoted - theo
        })

df_hw1 = pd.DataFrame(records)
display(df_hw1.head())
fig = px.bar(df_hw1, x='Tenor', y='Deviation', color=df_hw1['Date'].dt.year.astype(str), barmode='group', title='Quoted vs theoretical forward deviation')
fig.show()
worst = df_hw1.loc[df_hw1['Deviation'].abs().idxmax()]
worse_summary = worst
worse_summary


,Date,Tenor,Theoretical,Quoted,Deviation
0,2005-10-03,1M,1.194095,1.19425,0.000155
1,2005-10-03,3M,1.197987,1.19812,0.000133
2,2005-10-03,6M,1.204454,1.20462,0.000166
3,2005-10-03,1Y,1.217417,1.21789,0.000473
4,2006-10-02,1M,1.276000,1.27619,0.000190


Date           2008-10-01 00:00:00
Tenor                           1Y
Theoretical               1.382663
Quoted                     1.39603
Deviation                 0.013367
Name: 15, dtype: object

## Homework 2 – Forward Arbitrage and Commodity Futures

In [4]:

# Forward arbitrage roll-down
hw2_path = Path('Assignments/Assignment 2/DataHW2_2024.xls')
arb = pd.read_excel(hw2_path, sheet_name='ForwardArbitrage', header=2)
arb.columns = ['Date','MaturityYears','Spot','Fwd1M','Fwd3M','Fwd6M','Fwd1Y','US1M','US3M','US6M','US1Y','EU1M','EU3M','EU6M','EU1Y']
arb['Date'] = pd.to_datetime(arb['Date'], errors='coerce')
arb = arb.dropna(subset=['Date'])
initial = arb.iloc[0]
remaining6m = arb.iloc[1]

k_forward = initial['Fwd1Y']
spot0 = initial['Spot']
r_us0 = libor_simple_to_continuous(initial['US1Y'],1)
r_eu0 = libor_simple_to_continuous(initial['EU1Y'],1)
f_fair0 = forward_fx(spot0, r_us0, r_eu0, 1)
print(f"Initial quoted 1Y forward {k_forward:.4f} vs fair {f_fair0:.4f}")

spot_t = remaining6m['Spot']
t_remain = remaining6m['MaturityYears']
r_us_t = libor_simple_to_continuous(remaining6m['US6M'], t_remain)
r_eu_t = libor_simple_to_continuous(remaining6m['EU6M'], t_remain)
f_fair_t = forward_fx(spot_t, r_us_t, r_eu_t, t_remain)
value_short = (k_forward - f_fair_t) * math.exp(-r_us_t * t_remain)
print(f"Value of short forward after 6M: {value_short:.6f} USD per EUR notional")

borrow_leg = -spot0 * math.exp(r_us0*1)
invest_leg = spot0/spot0 * math.exp(r_eu0*1)
dollar_cost = borrow_leg * math.exp(-r_us_t*t_remain)
dollar_asset = invest_leg * spot_t
synthetic_value = dollar_asset + dollar_cost
print(f"Synthetic long forward PV at 6M: {synthetic_value:.6f}")


Initial quoted 1Y forward 1.3960 vs fair 1.3827
Value of short forward after 6M: 0.071791 USD per EUR notional
Synthetic long forward PV at 6M: -0.050268


/tmp/ipykernel_12291/1888150584.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [5]:

# Southwest hedge analysis using futures and jet fuel prices
sheet='Light Crude fut. prices'
raw_prices = pd.read_excel(hw2_path, sheet_name=sheet, header=None)
header_row = 3
cols = raw_prices.iloc[header_row].tolist()
data = raw_prices.iloc[header_row+1:, :len(cols)].copy()
data.columns = cols
fuel_col = 'Fuel price per gallon'
data = data.dropna(subset=['Day']).copy()
data['Day'] = pd.to_datetime(data['Day'], errors='coerce')
for c in ['FEB. 08','MAR. 08','APR. 08', fuel_col]:
    data[c] = pd.to_numeric(data[c], errors='coerce')
hedge_slice = data.dropna(subset=['FEB. 08','MAR. 08','APR. 08'])

contracts_per_month = {
    'JAN': ('FEB. 08','2008-01-22'),
    'FEB': ('MAR. 08','2008-02-20'),
    'MAR': ('APR. 08','2008-03-20'),
}
consumption_gal = 1511e6/4
barrels_per_month = consumption_gal/3/42
hedge_ratio = 0.75
pnl_rows = []

for month,(contract, settle_date) in contracts_per_month.items():
    settle_date = pd.to_datetime(settle_date)
    trade_row = hedge_slice.iloc[hedge_slice['Day'].sub(pd.to_datetime('2007-12-31')).abs().idxmin()]
    settle_row = hedge_slice.iloc[hedge_slice['Day'].sub(settle_date).abs().idxmin()]
    price_on_trade = trade_row[contract]
    settle_price = settle_row[contract]
    jet_price = settle_row[fuel_col]
    contracts_needed = hedge_ratio * barrels_per_month
    pnl_per_barrel = settle_price - price_on_trade
    total_pnl = pnl_per_barrel * contracts_needed * 1000
    implicit_price = jet_price - total_pnl/(contracts_needed*1000*42)
    pnl_rows.append({'Month':month,'FuturesStart':price_on_trade,'FuturesSettle':settle_price,'JetFuel':jet_price,'Contracts':contracts_needed,'TotalPnL':total_pnl,'ImplicitJetPrice':implicit_price})

hedge_df = pd.DataFrame(pnl_rows)
display(hedge_df)
fig = px.line(hedge_slice, x='Day', y=['FEB. 08','MAR. 08','APR. 08',fuel_col], title='Crude futures vs jet fuel')
fig.show()


,Month,FuturesStart,FuturesSettle,JetFuel,Contracts,TotalPnL,ImplicitJetPrice
0,JAN,95.09,145.29,4.223667,2.248512e+06,1.128753e+11,3.028429
1,FEB,94.90,145.86,4.223667,2.248512e+06,1.145842e+11,3.010333
2,MAR,94.53,146.12,4.223667,2.248512e+06,1.160007e+11,2.995333


## Homework 3 – Currency Swaps and Jet Fuel Options

In [6]:

# Currency swap fair rate
zcb = pd.read_excel('Assignments/Assignment 3/Greece_GS_table1.xls', sheet_name='Sheet1', header=None, names=['blank','T','ZEU','ZUS'])
zcb = zcb.dropna(subset=['T'])
spot = 0.8475
notional_usd = 50
notional_eur = 59
coupon_usd = 0.06
freq=2

times = [i/freq for i in range(1, 10*freq+1)]
zeu = dict(zip(zcb['T'], zcb['ZEU']))
zus = dict(zip(zcb['T'], zcb['ZUS']))
pv_eur = sum(zeu[t] for t in times)
pv_usd_leg = sum(coupon_usd/freq*notional_usd*zus[t]*spot for t in times) + notional_usd*spot*zus[10]
rate_eur = pv_usd_leg/(notional_eur*(pv_eur))
print(f"Implied EUR fixed rate: {rate_eur*freq:.4%}")

spot_hist = 0.8148
quoted_rate = 0.07
pv_eur_fixed = notional_eur*quoted_rate/freq*pv_eur + notional_eur*zeu[10]
pv_usd_float = pv_usd_leg * (spot_hist/spot)
value_gs = pv_usd_float - pv_eur_fixed
print(f"Goldman swap PV (USD perspective): {value_gs:.2f}")


Implied EUR fixed rate: 9.5825%
Goldman swap PV (USD perspective): -25.41


In [7]:

# Option hedging counts for jet fuel
option_table = pd.DataFrame({
    'Strike':[60,65,70,75,80,85,88.61,90,95,100,105,110,115,120,125,130],
    'Call':[35674,30744,25859,21098,16593,12513,9923,9016,6195,4060,2541,1522,875,485,259,135],
    'Put':[3,17,76,259,698,1562,2541,3009,5132,7941,11366,15292,19589,24142,28861,33680]
})
consumption_gal = 1511e6/4
barrels = consumption_gal/42
hedge_ratio = 0.75
calls_needed = hedge_ratio*barrels/1000
print(f"105-strike calls needed: {calls_needed:,.0f}")
call_price = option_table.loc[option_table['Strike']==105,'Call'].item()
option_table['ZeroCostPutCount'] = (calls_needed*call_price)/option_table['Put']
option_table[['Strike','Put','ZeroCostPutCount']].head()


105-strike calls needed: 6,746


,Strike,Put,ZeroCostPutCount
0,60.0,3,5.713469e+06
1,65.0,17,1.008259e+06
2,70.0,76,2.255317e+05
3,75.0,259,6.617917e+04
4,80.0,698,2.455646e+04


## Homework 4 – Barings Straddle and Binomial Trees

In [8]:

# Leeson short straddle payoff
k = 19750
premium_call = 9.90
premium_put = 9.80
contract_size = 10000
prices = np.linspace(12000, 26000, 50)
profit = []
for s in prices:
    payoff = -max(s-k,0) - max(k-s,0) + (premium_call + premium_put)
    profit.append(payoff*contract_size)
fig = go.Figure(data=go.Scatter(x=prices, y=profit))
fig.update_layout(title='Short straddle profit (per contract)', xaxis_title='Nikkei', yaxis_title='JPY')
fig.show()

s_final = 17473
per_contract = -max(s_final-k,0) - max(k-s_final,0) + (premium_call+premium_put)
total_loss = per_contract*contract_size*35500
print(f"Total loss on options: {total_loss:,.0f} JPY")


Total loss on options: -801,341,500,000 JPY


## Homework 5 – Binomial Option Pricing for FDA Example

In [9]:

S_u = 21
S_d = 10
q = 0.7
beta = 2
r_cc = 0.05
market_rp = 0.0644
r_disc = math.exp(r_cc) - 1
expected_ret = r_disc + beta*market_rp
S0 = (q*S_u + (1-q)*S_d)/(1+expected_ret)
print(f"Implied S0 from CAPM: {S0:.4f}")

u = S_u/S0
d = S_d/S0
p_star = (math.exp(r_cc) - d)/(u-d)
call_payoff_u = max(S_u - S0,0)
call_payoff_d = max(S_d - S0,0)
call_val = math.exp(-r_cc)*(p_star*call_payoff_u + (1-p_star)*call_payoff_d)
print(f"ATM call value (1-yr): {call_val:.4f}")
put_val = math.exp(-r_cc)*(p_star*max(0,S0-S_u)+(1-p_star)*max(0,S0-S_d))
print(f"ATM put value: {put_val:.4f}")


Implied S0 from CAPM: 14.9991
ATM call value (1-yr): 2.9933
ATM put value: 2.2617


## Homework 6 – Structured Products (PLUS)

In [10]:

notional = 10
leverage = 3.0
cap = 11.90
s0 = 1329.51
s_range = np.linspace(0.6*s0, 1.4*s0, 50)
payoff = []
for s in s_range:
    pct = (s - s0)/s0
    if s > s0:
        amt = min(notional + leverage*notional*pct, cap)
    else:
        amt = notional*(s/s0)
    payoff.append(amt)
fig = go.Figure(go.Scatter(x=s_range, y=payoff))
fig.update_layout(title='PLUS payoff profile', xaxis_title='S&P 500 at maturity', yaxis_title='Payment ($)')
fig.show()


## Homework 7 – American Options and KMV Default

In [11]:

S0 = 100
u = 1.1
d = 1/u
p = 0.6
r = 0.02
K = 100
T = 3
nodes = {(0,0): S0}
for t in range(1,T+1):
    for i in range(t+1):
        s = S0*(u**i)*(d**(t-i))
        nodes[(t,i)] = s

call = {}
put = {}
for i in range(T+1):
    s = nodes[(T,i)]
    call[(T,i)] = max(s-K,0)
    put[(T,i)] = max(K-s,0)

disc = math.exp(-r)
for t in reversed(range(T)):
    for i in range(t+1):
        s = nodes[(t,i)]
        call_cont = disc*(p*call[(t+1,i+1)] + (1-p)*call[(t+1,i)])
        put_cont = disc*(p*put[(t+1,i+1)] + (1-p)*put[(t+1,i)])
        call[(t,i)] = max(call_cont, s-K)
        put[(t,i)] = max(put_cont, K-s)

print(f"American call value: {call[(0,0)]:.4f}")
print(f"American put value: {put[(0,0)]:.4f}")

balance = pd.read_excel('Assignments/Assignment 7/HW7_Data.xls', sheet_name='BalanceSheet', header=3)
item_col = balance.columns[0]
citi_col = balance.columns[2]
citi = balance.set_index(item_col)[citi_col]
assets = citi.get('Total assets', citi.dropna().iloc[-1])
deposits = citi.get('Deposits', 0)
long_term = citi.get('Long term', 0)
default_point = deposits + 0.5*long_term
asset_vol = 0.3
asset_value = assets
DD = (math.log(asset_value/default_point) + (r - 0.5*asset_vol**2))/(asset_vol)
default_prob = 1 - 0.5*(1+math.erf(DD/math.sqrt(2)))
print(f"Distance to default: {DD:.2f}; 1y PD ~ {default_prob:.2%}")


American call value: 10.8017
American put value: 4.3863
Distance to default: 2.38; 1y PD ~ 0.86%
